# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120607e+02     1.587999e+00
 * time: 0.7168409824371338
     1     1.108765e+01     9.762699e-01
 * time: 3.0939249992370605
     2    -1.209760e+01     9.864028e-01
 * time: 3.722234010696411
     3    -3.446767e+01     7.418627e-01
 * time: 4.690423965454102
     4    -4.816421e+01     5.122665e-01
 * time: 5.660660982131958
     5    -5.705984e+01     1.866882e-01
 * time: 6.591387987136841
     6    -5.984195e+01     1.022680e-01
 * time: 7.224095106124878
     7    -6.086382e+01     5.829394e-02
 * time: 7.855782985687256
     8    -6.128192e+01     7.181178e-02
 * time: 8.5053071975708
     9    -6.160855e+01     3.460005e-02
 * time: 9.130141973495483
    10    -6.181542e+01     2.566079e-02
 * time: 9.758421182632446
    11    -6.196732e+01     2.217148e-02
 * time: 10.399819135665894
    12    -6.204696e+01     2.284935e-02
 * time: 11.04395604133606
    13    -6.211682e+01     1.798849e-02
 * time: 11.664241075515747
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557689
    AtomicNonlocal      14.8522651
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336822

    total               -62.261666458792
